In [53]:
import gym
from gym import spaces
import random

In [54]:
MAX_ACCOUNT_BALANCE = 2147483647
MAX_NUM_SHARES = 2147483647
MAX_SHARE_PRICE = 5000
MAX_OPEN_POSITIONS = 5
MAX_STEPS = 20000

INITIAL_ACCOUNT_BALANCE = 10000

In [55]:
class StockTradingEnv(gym.Env):
    def __init__(self, df):
        super(StockTradingEnv, self).__init__()
        self.df = df
        self.reward_range = (0, MAX_ACCOUNT_BALANCE)
        self.action_space = spaces.Box(low=np.array([0, 0]), high=np.array([3, 1]), dtype=np.float16) # buy, sell, hold actions
        self.observation_space = spaces.Box(low=0, high=1, shape=(6, 6), dtype=np.float16)
        
    def step(self, action):
        self._take_action(action)
        self.current_step+=1
        if self.current_step > len(self.df.loc[:, 'Open'].values) - 6:
            self.current_step = 0
        delay_modifier = (self.current_step / MAX_STEPS)
        reward = self.balance * delay_modifier
        done = self.net_worth <= 0  
        obs = self._next_observation()  
        return obs, reward, done, {}
    
    def reset(self):
        self.balance = INITIAL_ACCOUNT_BALANCE
        self.net_worth = INITIAL_ACCOUNT_BALANCE
        self.max_net_worth = INITIAL_ACCOUNT_BALANCE
        self.shares_held = 0
        self.cost_basis = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0
        
        self.current_step = random.randint(0, len(self.df.loc[:, 'Open'].values-6))
        return self._next_observation()
    
    def _take_action(self, action):
        current_price = random.uniform(self.df.loc[self.current_step, 'Open'], self.df.loc[self.current_step, 'Close'])
        action_type=action[0]
        amount = action[1]
        if action_type < 1:
            # Buy amount % of balance in shares
            total_possible = self.balance/current_price
            shares_bought = total_possible * amount
            prev_cost = self.cost_basis * self.shares_held
            additional_cost = shares_bought * current_price    
            self.balance -= additional_cost
            self.cost_basis = (prev_cost + additional_cost) / (self.shares_held + shares_bought)
            self.shares_held += shares_bought
        elif action_type < 2:
            # Sell amount % of shares held
            shares_sold = self.shares_held * amount 
            self.balance += shares_sold * current_price
            self.shares_held -= shares_sold
            self.total_shares_sold += shares_sold
            self.total_sales_value += shares_sold * current_price
            
        self.net_worth = self.balance + self.shares_held * current_price
        if self.net_worth > self.max_net_worth:
            self.max_net_worth = self.net_worth 
            
        if self.shares_held == 0:
            self.cost_basis = 0
            
    
    def _next_observation(self):
        # Get the data points for the last 5 days and scale to between 0-1
        frame = np.array([self.df.loc[self.current_step: self.current_step + 5, 'Open'].values / MAX_SHARE_PRICE,
                          self.df.loc[self.current_step: self.current_step + 5, 'High'].values / MAX_SHARE_PRICE,
                          self.df.loc[self.current_step: self.current_step + 5, 'Low'].values / MAX_SHARE_PRICE,
                          self.df.loc[self.current_step: self.current_step + 5, 'Close'].values / MAX_SHARE_PRICE,
                          self.df.loc[self.current_step: self.current_step + 5, 'Volume'].values / MAX_NUM_SHARES,
                         ])
        # Append additional data and scale each value to between 0-1
        obs = np.append(frame, [[self.balance / MAX_ACCOUNT_BALANCE, self.max_net_worth / MAX_ACCOUNT_BALANCE,
                                 self.shares_held / MAX_NUM_SHARES, self.cost_basis / MAX_SHARE_PRICE, 
                                 self.total_shares_sold / MAX_NUM_SHARES, 
                                 self.total_sales_value / (MAX_NUM_SHARES * MAX_SHARE_PRICE),
                                ]], axis=0)
        return obs
    
    def render(self, mode='human', close=False):
        # Render the environment to the screen
        profit = self.net_worth - INITIAL_ACCOUNT_BALANCE  
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance}')
        print(f'Shares held: {self.shares_held} Total sold: {self.total_shares_sold})')
        print(f'Avg cost for held shares: {self.cost_basis} (Total sales value: {self.total_sales_value})')
        print(f'Net worth: {self.net_worth} (Max net worth: {self.max_net_worth})')
        print(f'Profit: {profit}')

In [56]:
import gym
import json
import datetime as dt
import pandas as pd
import numpy as np

In [57]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

In [58]:
df = pd.read_csv('./data/AAPL.csv')
df = df.sort_values('Date')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-29,100.589996,100.690002,98.040001,99.620003,93.514290,142718700
1,2014-10-06,99.949997,102.379997,98.309998,100.730003,94.556244,280258200
2,2014-10-13,101.330002,101.779999,95.180000,97.669998,91.683792,358539800
3,2014-10-20,98.320000,105.489998,98.220001,105.220001,98.771042,358532900
4,2014-10-27,104.849998,108.040001,104.699997,108.000000,101.380676,220230600


In [59]:
# The algorithms require a vectorized environment to run
env = DummyVecEnv([lambda: StockTradingEnv(df)])
model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=20000)
obs = env.reset()
for i in range(2000):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()

C:\Users\User\Anaconda3\envs\tf_v1_env\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float16
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
C:\Users\User\Anaconda3\envs\tf_v1_env\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in double_scalars


---------------------------------------
| approxkl           | 4.3259643e-06  |
| clipfrac           | 0.0            |
| explained_variance | -0.000202      |
| fps                | 331            |
| n_updates          | 1              |
| policy_entropy     | 2.836316       |
| policy_loss        | -0.00030112558 |
| serial_timesteps   | 128            |
| time_elapsed       | 0              |
| total_timesteps    | 128            |
| value_loss         | 2137.852       |
---------------------------------------
--------------------------------------
| approxkl           | 1.273788e-05  |
| clipfrac           | 0.0           |
| explained_variance | -0.000128     |
| fps                | 812           |
| n_updates          | 2             |
| policy_entropy     | 2.834731      |
| policy_loss        | -0.0006130049 |
| serial_timesteps   | 256           |
| time_elapsed       | 0.387         |
| total_timesteps    | 256           |
| value_loss         | 356.12216     |
------------

--------------------------------------
| approxkl           | 0.0011423399  |
| clipfrac           | 0.001953125   |
| explained_variance | 9.76e-05      |
| fps                | 883           |
| n_updates          | 18            |
| policy_entropy     | 2.8628314     |
| policy_loss        | -0.0037063025 |
| serial_timesteps   | 2304          |
| time_elapsed       | 2.75          |
| total_timesteps    | 2304          |
| value_loss         | 16238.298     |
--------------------------------------
--------------------------------------
| approxkl           | 0.00096758513 |
| clipfrac           | 0.001953125   |
| explained_variance | -0.00231      |
| fps                | 871           |
| n_updates          | 19            |
| policy_entropy     | 2.8663805     |
| policy_loss        | -0.00696027   |
| serial_timesteps   | 2432          |
| time_elapsed       | 2.9           |
| total_timesteps    | 2432          |
| value_loss         | 65061.13      |
-------------------------

--------------------------------------
| approxkl           | 0.00028875133 |
| clipfrac           | 0.0           |
| explained_variance | 0.00016       |
| fps                | 893           |
| n_updates          | 35            |
| policy_entropy     | 2.8601756     |
| policy_loss        | 0.0024911407  |
| serial_timesteps   | 4480          |
| time_elapsed       | 5.21          |
| total_timesteps    | 4480          |
| value_loss         | 490634.53     |
--------------------------------------
-------------------------------------
| approxkl           | 0.0021682954 |
| clipfrac           | 0.001953125  |
| explained_variance | -0.000267    |
| fps                | 889          |
| n_updates          | 36           |
| policy_entropy     | 2.8602467    |
| policy_loss        | -0.002140331 |
| serial_timesteps   | 4608         |
| time_elapsed       | 5.36         |
| total_timesteps    | 4608         |
| value_loss         | 1772553.1    |
-------------------------------------

--------------------------------------
| approxkl           | 0.00072609656 |
| clipfrac           | 0.0           |
| explained_variance | -0.000152     |
| fps                | 911           |
| n_updates          | 52            |
| policy_entropy     | 2.8506258     |
| policy_loss        | -0.002155886  |
| serial_timesteps   | 6656          |
| time_elapsed       | 7.66          |
| total_timesteps    | 6656          |
| value_loss         | 1063984.9     |
--------------------------------------
--------------------------------------
| approxkl           | 0.00059852866 |
| clipfrac           | 0.0           |
| explained_variance | 5.85e-05      |
| fps                | 917           |
| n_updates          | 53            |
| policy_entropy     | 2.8513243     |
| policy_loss        | 0.0027296436  |
| serial_timesteps   | 6784          |
| time_elapsed       | 7.81          |
| total_timesteps    | 6784          |
| value_loss         | 884704.8      |
-------------------------

...

Step: 148
Balance: 10026.466517743604
Shares held: 9.992127772065029 Total sold: 564.0776708001649)
Avg cost for held shares: 97.9022315719053 (Total sales value: 67816.49550550539)
Net worth: 11529.575093641217 (Max net worth: 12903.222420659582)
Profit: 1529.5750936412169
Step: 149
Balance: 10026.466517743604
Shares held: 9.992127772065029 Total sold: 564.0776708001649)
Avg cost for held shares: 97.9022315719053 (Total sales value: 67816.49550550539)
Net worth: 11548.523513993525 (Max net worth: 12903.222420659582)
Profit: 1548.5235139935248
Step: 150
Balance: 10026.466517743604
Shares held: 9.992127772065029 Total sold: 564.0776708001649)
Avg cost for held shares: 97.9022315719053 (Total sales value: 67816.49550550539)
Net worth: 11595.945062356048 (Max net worth: 12903.222420659582)
Profit: 1595.9450623560479
Step: 151
Balance: 10026.466517743604
Shares held: 9.992127772065029 Total sold: 564.0776708001649)
Avg cost for held shares: 97.9022315719053 (Total sales value: 67816.495505

Step: 2
Balance: 10026.466517743604
Shares held: 9.992127772065029 Total sold: 564.0776708001649)
Avg cost for held shares: 97.9022315719053 (Total sales value: 67816.49550550539)
Net worth: 11025.527877540255 (Max net worth: 12903.222420659582)
Profit: 1025.5278775402548
Step: 3
Balance: 10026.466517743604
Shares held: 9.992127772065029 Total sold: 564.0776708001649)
Avg cost for held shares: 97.9022315719053 (Total sales value: 67816.49550550539)
Net worth: 11008.995201018724 (Max net worth: 12903.222420659582)
Profit: 1008.9952010187244
Step: 4
Balance: 10026.466517743604
Shares held: 9.992127772065029 Total sold: 564.0776708001649)
Avg cost for held shares: 97.9022315719053 (Total sales value: 67816.49550550539)
Net worth: 11059.922099718733 (Max net worth: 12903.222420659582)
Profit: 1059.922099718733
Step: 5
Balance: 10026.466517743604
Shares held: 9.992127772065029 Total sold: 564.0776708001649)
Avg cost for held shares: 97.9022315719053 (Total sales value: 67816.49550550539)
Ne

Step: 1
Balance: 11141.218406018905
Shares held: 0.0 Total sold: 574.06979857223)
Avg cost for held shares: 0 (Total sales value: 68931.2473937807)
Net worth: 11141.218406018905 (Max net worth: 12903.222420659582)
Profit: 1141.2184060189047
Step: 2
Balance: 11141.218406018905
Shares held: 0.0 Total sold: 574.06979857223)
Avg cost for held shares: 0 (Total sales value: 68931.2473937807)
Net worth: 11141.218406018905 (Max net worth: 12903.222420659582)
Profit: 1141.2184060189047
Step: 3
Balance: 11141.218406018905
Shares held: 0.0 Total sold: 574.06979857223)
Avg cost for held shares: 0 (Total sales value: 68931.2473937807)
Net worth: 11141.218406018905 (Max net worth: 12903.222420659582)
Profit: 1141.2184060189047
Step: 4
Balance: 11141.218406018905
Shares held: 0.0 Total sold: 574.06979857223)
Avg cost for held shares: 0 (Total sales value: 68931.2473937807)
Net worth: 11141.218406018905 (Max net worth: 12903.222420659582)
Profit: 1141.2184060189047
Step: 5
Balance: 11141.218406018905


Step: 35
Balance: 8608.181246658805
Shares held: 18.035725771905877 Total sold: 633.580323240194)
Avg cost for held shares: 143.99319818070364 (Total sales value: 76440.11314004629)
Net worth: 10967.175060854244 (Max net worth: 12903.222420659582)
Profit: 967.1750608542443
Step: 36
Balance: 8608.181246658805
Shares held: 18.035725771905877 Total sold: 633.580323240194)
Avg cost for held shares: 143.99319818070364 (Total sales value: 76440.11314004629)
Net worth: 10934.310594434632 (Max net worth: 12903.222420659582)
Profit: 934.3105944346316
Step: 37
Balance: 8608.181246658805
Shares held: 18.035725771905877 Total sold: 633.580323240194)
Avg cost for held shares: 143.99319818070364 (Total sales value: 76440.11314004629)
Net worth: 10926.54006697957 (Max net worth: 12903.222420659582)
Profit: 926.5400669795708
Step: 38
Balance: 8608.181246658805
Shares held: 18.035725771905877 Total sold: 633.580323240194)
Avg cost for held shares: 143.99319818070364 (Total sales value: 76440.1131400462

Step: 66
Balance: 10292.107737299926
Shares held: 3.528838016415296 Total sold: 648.0872109956846)
Avg cost for held shares: 143.99319818070364 (Total sales value: 78124.0396306874)
Net worth: 10669.430642550078 (Max net worth: 12903.222420659582)
Profit: 669.4306425500781
Step: 67
Balance: 10292.107737299926
Shares held: 3.528838016415296 Total sold: 648.0872109956846)
Avg cost for held shares: 143.99319818070364 (Total sales value: 78124.0396306874)
Net worth: 10645.912119169725 (Max net worth: 12903.222420659582)
Profit: 645.912119169725
Step: 68
Balance: 10292.107737299926
Shares held: 3.528838016415296 Total sold: 648.0872109956846)
Avg cost for held shares: 143.99319818070364 (Total sales value: 78124.0396306874)
Net worth: 10638.057577496158 (Max net worth: 12903.222420659582)
Profit: 638.057577496158
Step: 69
Balance: 10292.107737299926
Shares held: 3.528838016415296 Total sold: 648.0872109956846)
Avg cost for held shares: 143.99319818070364 (Total sales value: 78124.0396306874